In [1]:
from llama_hub.llama_packs.tables.chain_of_table.base import (
    ChainOfTableQueryEngine,
    serialize_table
)

In [2]:
import pandas as pd

df = pd.read_csv("~/Downloads/WikiTableQuestions/csv/200-csv/11.csv")

In [3]:
df

,Award,Category,Nominee,Result
0,"Academy Awards, 1972",Best Picture,Phillip D'Antoni,Won
1,"Academy Awards, 1972",Best Director,William Friedkin,Won
2,"Academy Awards, 1972",Best Actor,Gene Hackman,Won
3,"Academy Awards, 1972",Best Adapted Screenplay,Ernest Tidyman,Won
4,"Academy Awards, 1972",Film Editing,Gerald B. Greenberg,Won
5,"Academy Awards, 1972",Best Supporting Actor,Roy Scheider,Nominated
6,"Academy Awards, 1972",Best Cinematography,Owen Roizman,Nominated
7,"Academy Awards, 1972",Best Sound,Theodore Soderberg\nChristopher Newman,Nominated
8,"American Cinema Editors, 1972",Best Edited Feature Film,Gerald B. Greenberg,Nominated
9,"BAFTA, 1972",Best Actor,Gene Hackman,Won


In [4]:
print(serialize_table(df))

col : Award | Category | Nominee | Result
row 1 : Academy Awards, 1972 | Best Picture | Phillip D'Antoni | Won
row 2 : Academy Awards, 1972 | Best Director | William Friedkin | Won
row 3 : Academy Awards, 1972 | Best Actor | Gene Hackman | Won
row 4 : Academy Awards, 1972 | Best Adapted Screenplay | Ernest Tidyman | Won
row 5 : Academy Awards, 1972 | Film Editing | Gerald B. Greenberg | Won
row 6 : Academy Awards, 1972 | Best Supporting Actor | Roy Scheider | Nominated
row 7 : Academy Awards, 1972 | Best Cinematography | Owen Roizman | Nominated
row 8 : Academy Awards, 1972 | Best Sound | Theodore Soderberg\nChristopher Newman | Nominated
row 9 : American Cinema Editors, 1972 | Best Edited Feature Film | Gerald B. Greenberg | Nominated
row 10 : BAFTA, 1972 | Best Actor | Gene Hackman | Won
row 11 : BAFTA, 1972 | Best Film Editing | Gerald B. Greenberg | Won
row 12 : BAFTA, 1972 | Best Direction | William Friedkin | Nominated
row 13 : BAFTA, 1972 | Best Film | Philip D'Antoni | Nominate

In [5]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-4-1106-preview")

In [6]:
query_engine = ChainOfTableQueryEngine(df, llm=llm)

In [7]:
# query_engine.query("Which of the months between june and august have the highest average high temperature?")
# response = query_engine.query("Which of these people disappeared in January?")
# response = query_engine.query("Which of these works won a BAFTA award? (film or tv)")
response = query_engine.query("Who won best Director in the 1972 Academy Awards?")

ITERATION
col : Award | Category | Nominee | Result
row 1 : Academy Awards, 1972 | Best Picture | Phillip D'Antoni | Won
row 2 : Academy Awards, 1972 | Best Director | William Friedkin | Won
row 3 : Academy Awards, 1972 | Best Actor | Gene Hackman | Won
row 4 : Academy Awards, 1972 | Best Adapted Screenplay | Ernest Tidyman | Won
row 5 : Academy Awards, 1972 | Film Editing | Gerald B. Greenberg | Won
row 6 : Academy Awards, 1972 | Best Supporting Actor | Roy Scheider | Nominated
row 7 : Academy Awards, 1972 | Best Cinematography | Owen Roizman | Nominated
row 8 : Academy Awards, 1972 | Best Sound | Theodore Soderberg\nChristopher Newman | Nominated
row 9 : American Cinema Editors, 1972 | Best Edited Feature Film | Gerald B. Greenberg | Nominated
row 10 : BAFTA, 1972 | Best Actor | Gene Hackman | Won
row 11 : BAFTA, 1972 | Best Film Editing | Gerald B. Greenberg | Won
row 12 : BAFTA, 1972 | Best Direction | William Friedkin | Nominated
row 13 : BAFTA, 1972 | Best Film | Philip D'Antoni 

In [12]:
str(response.response)

'assistant: William Friedkin.'